In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

from fastai.vision import *
from pathlib import Path
import pandas as pd
path=Path('/content/drive/My Drive/Colab Notebooks/bbc_train/')

In [0]:
images, lbl_bbox = get_annotations(path/'bboxes.json')
img2bbox = dict(zip(images, lbl_bbox))
get_y_func = lambda o:img2bbox[o.name]
src = ObjectItemList.from_folder(path/'images')
def get_data(bs, size):
    src = ObjectItemList.from_folder(path/'images')
    src = src.split_by_rand_pct(0.2)
    src = src.label_from_func(get_y_func)
    src = src.transform(get_transforms(max_rotate=0), size=size, tfm_y=True)
    return src.databunch(path=path, bs=bs, collate_fn=bb_pad_collate)

data = get_data(bs=64,size=256)

In [0]:

class MyLoss(nn.Module):
    def forward(self, yhat, bbox_tgts, class_tgts):
        det_loss=nn.L1Loss()(yhat[:,:4].unsqueeze_(dim=1), bbox_tgts)
        cls_loss=nn.CrossEntropyLoss()(yhat[:,4:], class_tgts.view(-1))
        #print(det_loss, cls_loss)
        
        return det_loss + 1.0*cls_loss

def my_accuracy(yhat, bbox_tgts, class_tgts):
    return accuracy(yhat[:,4:], class_tgts.view(-1))

def my_l1(yhat, bbox_tgts, class_tgts):
    return nn.L1Loss()(yhat[:,:4].unsqueeze_(dim=1), bbox_tgts)

In [0]:
custom_head = nn.Sequential(
    AdaptiveConcatPool2d((4,4)),
    Flatten(),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(16384,256), #2*512*4*4
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),
    nn.Linear(256,8))

In [0]:

learn=cnn_learner(data, models.resnet34, custom_head=custom_head, loss_func=MyLoss(), metrics=[my_accuracy, my_l1])

In [0]:
learn.freeze()

In [9]:

lr=1e-2
learn.fit_one_cycle(20, slice(lr/10, lr))

epoch,train_loss,valid_loss,my_accuracy,my_l1,time
0,2.616735,2.004865,0.819149,1.378592,00:21
1,2.028011,1.480738,0.920213,1.119667,00:20
2,1.699899,1.144495,0.882979,0.736329,00:19
3,1.401936,0.671724,0.957447,0.463895,00:20
4,1.163221,0.453575,0.984043,0.296333,00:20
5,0.959676,0.417631,0.962766,0.211363,00:20
6,0.800018,0.300210,0.989362,0.178495,00:20
7,0.674214,0.298120,0.994681,0.200982,00:20
8,0.578039,0.262189,0.973404,0.134209,00:20
9,0.500729,0.246598,0.989362,0.146787,00:20


In [0]:
learn.unfreeze()

In [11]:

lr=5e-4
learn.fit_one_cycle(20, slice(lr/10, lr))

epoch,train_loss,valid_loss,my_accuracy,my_l1,time
0,0.139556,0.190926,0.994681,0.084292,00:21
1,0.136073,0.181262,0.994681,0.078254,00:21
2,0.135246,0.163591,0.989362,0.079625,00:21
3,0.135428,0.184923,0.984043,0.069863,00:21
4,0.142508,0.374403,0.968085,0.094130,00:21
5,0.143282,0.332440,0.962766,0.082189,00:20
6,0.156606,0.846012,0.888298,0.108878,00:21
7,0.152081,0.217722,0.989362,0.075441,00:21
8,0.143197,0.137959,0.994681,0.059539,00:21
9,0.132538,0.127894,0.994681,0.053429,00:21


In [0]:
learn.export()

In [0]:
learn.path